In [55]:
import numpy as np
"""
  7 % ================
  8 % top_pars: A set of parameters describing the topology of the network. Different choices exists:
  9 %   * top_pars=[1 xmin xmax ymin ymax N seed]: All N nodes are placed randomly on an othogonal. 
 10 %   * top_pars=[2 xmin xmax ymin ymax N1 N2 seed side]: All N1*N2 nodes are placed on a grid in an orthogonal. 
 11 %     The grid locations are randomly perturbed so that nodes are placed within a square of side side centered 
 12 %     at the grid location. 
 13 %   In both cases, the orthogonal is specified by the arguments xmin, xmax, ymin, ymax, and the seed used for
 14 %   the random number generator is the argument seed. 
 15 % channel_pars: A set of parameters describing the properties of the channel. Different choices exist:
 16 %   * channel_pars=[1 alpha]: The channel gain g decreases with distance d between nodes as follows: 
 17 %                        g=min(1,d^(-alpha))
         * channel_pars=[]
 18 % tx_pars: A set of parameters specifying the data rate achieved by a transmitter-receiver pair, as a
 19 %   function of received power. Different choices exist:
 20 %   * tx_pars=[1 W NP low_thres hi_thres unique]: The achieved data rate is 
 21 %            DR=0,                   if            W*log2(1+PR/(NP+PI))< low_thres,
 22 %            DR=W*log2(1+PR/(NP+PI), if low_thres<=W*log2(1+PR/(NP+PI))<=hi_thres,
 23 %            DR=hi_thres,            if hi_thres < W*log2(1+PR/(NP+PI),
 24 %     where PR is the power received from the transmitter and PI is the total power received from the 
 25 %     interferers. All transmitters transmit with unit power. If unique=1, then each node can participate in
 26 %     only a single transmission, as either a receiver or transmitter. Otherwise, no constraint is placed. 
 27 % TM_pars: A set of parameters specifying which transmission modes are considered:
 28 %   * TM_pars=[1 b]: Only TMs with b-strong primal rate vectors will be calculated, i.e. vectors with positive  
 29 %     rates that are strictly greater than the value b.
 30 % verbosity: A set of parameters specifying how much is printed or plotted by this routine as well as the
 31 %   routines involked by this routine. Fro this routine:
 32 %      * verbosity=0: nothing is plotted or printed, only output is produced. 
 33 %      * verbosity=1: In addition, we draw the network.
 34 %      * verbosity=2: In addition, we print intermediate results on groups of Transmission Modes processed. 
 35 %      * verbosity=3: In addition, we print intermediate results for individual TMs processed. 
 36 %      * verbosity=4: In addition, we pause after groups of TMs are proccessed.
 """
def create_model_net(top_pars, channel_pars, tx_pars, tm_pars):
    if top_pars[0] == 1:
        _, xmin, xmax, ymin, ymax, N, seed = top_pars
        np.random.seed(seed)
        mat = np.array([[xmax-xmin, 0 ], [0 ,ymax-ymin]]) 
        locations = np.ones((N,1)) @ np.array([xmin,ymin])[np.newaxis,...] + np.random.rand(N,2) @ mat
    
    else:
        _, xmin, xmax, ymin, ymax, N1, N2, seed, side = top_pars
        N = N1*N2
        np.random.seed(seed)
        pert=side*(np.random.rand(N1*N2,2)-0.5);
        xlocs = xmin+(np.arange(0,N1,1)/(N1+1))*(xmax-xmin);
        ylocs= ymin + (np.arange(0,N2,1)/(N2+1))*(ymax-ymin);
        for i in range(N1):
            for j in range(N2):
                locations[(i-1)*N2+j,:]=np.array([xlocs[i], ylocs[j]])+pert[(i-1)*N2+j,:];  
        
        
        
    _, a = channel_pars
    _, W, NP, low_thres, hi_thres, unique = tx_pars
    _,b = tm_pars
    
    # We calculate the gain matrix and the pairwise communication rates in the absense of interference
    G=np.zeros((N,N));
    R=np.zeros((N,N));
    for i in range(N):
        for j in range(i):
            dsquared=(locations[i,0]-locations[j,0])**2+(locations[i,1]-locations[j,1])**2;
            if channel_pars[0]==1:
               G[i,j]=min(1/(dsquared**(a/2)),1);
               G[j,i]=G[i,j];

            if tx_pars[0]==1:  # tx_pars=[1 W NP low_thres hi_thres unique]:
               R[i,j]=W*np.log2(1+G[i,j]/NP)
               print("R[i,j]" , R[i,j])
               print("low thres " , low_thres)
               if R[i,j]<low_thres:
                   print("Low threshold")
                   R[i,j]=0;
               elif R[i,j]> hi_thres: 
                   R[i,j]= hi_thres;
               R[j,i]=R[i,j];
    print("Gain matrix " , G)
    print("Rate matrix " , R)

    
    locations= np.zeros((N,2));
    return

In [57]:
# top_pars=[1 xmin xmax ymin ymax N seed]: A
# channel_pars=[1 alpha]
# * tx_pars=[1 W NP low_thres hi_thres unique]
# TM_pars=[1 b]: O
top_pars = [1 , 10 , 50 , 10 , 50 , 100 , 5 ]
channel_pars =[ 1 , 0.3]
tx_pars = [ 1, 50, 10, 0.5, 100, 1]
TM_pars = [ 1, 50]
create_model_net(top_pars, channel_pars, tx_pars, TM_pars)

R[i,j] 5.625061321693652
low thres  0.5
R[i,j] 3.1407296604527986
low thres  0.5
R[i,j] 3.0370397083880145
low thres  0.5
R[i,j] 2.6668059199382146
low thres  0.5
R[i,j] 2.620630368791794
low thres  0.5
R[i,j] 3.368139947135698
low thres  0.5
R[i,j] 2.621240054695412
low thres  0.5
R[i,j] 2.568235134549395
low thres  0.5
R[i,j] 2.939848931970832
low thres  0.5
R[i,j] 2.763866189952282
low thres  0.5
R[i,j] 3.8019619055281777
low thres  0.5
R[i,j] 3.6628453013628564
low thres  0.5
R[i,j] 3.0152216229218687
low thres  0.5
R[i,j] 2.585753723859738
low thres  0.5
R[i,j] 2.739359058692236
low thres  0.5
R[i,j] 2.5590607112469224
low thres  0.5
R[i,j] 2.510313711453196
low thres  0.5
R[i,j] 2.9576001072950757
low thres  0.5
R[i,j] 2.9045691828082063
low thres  0.5
R[i,j] 4.115903964178881
low thres  0.5
R[i,j] 2.625801631688705
low thres  0.5
R[i,j] 2.4301835841765236
low thres  0.5
R[i,j] 2.3958999412999167
low thres  0.5
R[i,j] 2.8501481276386698
low thres  0.5
R[i,j] 3.450386877249861
low

R[i,j] 2.502332688103861
low thres  0.5
R[i,j] 6.061511658756815
low thres  0.5
R[i,j] 2.8710047071767053
low thres  0.5
R[i,j] 2.5511404757908895
low thres  0.5
R[i,j] 2.4950716856219275
low thres  0.5
R[i,j] 2.8585642454992266
low thres  0.5
R[i,j] 3.5297005016403054
low thres  0.5
R[i,j] 2.985581182539786
low thres  0.5
R[i,j] 2.4425517616622043
low thres  0.5
R[i,j] 4.850811296642994
low thres  0.5
R[i,j] 3.589987989303049
low thres  0.5
R[i,j] 2.5005521243336415
low thres  0.5
R[i,j] 2.4058662442628913
low thres  0.5
R[i,j] 2.8815987757453017
low thres  0.5
R[i,j] 3.498477478088053
low thres  0.5
R[i,j] 3.4118908071651926
low thres  0.5
R[i,j] 3.576812149761366
low thres  0.5
R[i,j] 2.99077666142041
low thres  0.5
R[i,j] 2.6216961778226002
low thres  0.5
R[i,j] 3.0448104713941806
low thres  0.5
R[i,j] 2.615017600345222
low thres  0.5
R[i,j] 2.6046172495940256
low thres  0.5
R[i,j] 2.7874827497065757
low thres  0.5
R[i,j] 3.3146196171824744
low thres  0.5
R[i,j] 2.732451325057442
l

low thres  0.5
R[i,j] 2.611586648858559
low thres  0.5
R[i,j] 2.7752016694937787
low thres  0.5
R[i,j] 2.6898359393102056
low thres  0.5
R[i,j] 3.0301966732081347
low thres  0.5
R[i,j] 3.5074788765259814
low thres  0.5
R[i,j] 2.414443935690204
low thres  0.5
R[i,j] 2.643361784273716
low thres  0.5
R[i,j] 2.876149789861576
low thres  0.5
R[i,j] 3.71011483873544
low thres  0.5
R[i,j] 2.5081648450117298
low thres  0.5
R[i,j] 2.455667102110645
low thres  0.5
R[i,j] 4.355921748280183
low thres  0.5
R[i,j] 3.0874291479294174
low thres  0.5
R[i,j] 2.4186035500693848
low thres  0.5
R[i,j] 2.968175193177929
low thres  0.5
R[i,j] 2.6997323132930604
low thres  0.5
R[i,j] 2.614781692010258
low thres  0.5
R[i,j] 4.184463853711923
low thres  0.5
R[i,j] 2.9661183769116266
low thres  0.5
R[i,j] 2.76099198738491
low thres  0.5
R[i,j] 4.626875216588898
low thres  0.5
R[i,j] 2.5765998203827736
low thres  0.5
R[i,j] 3.6560584912989564
low thres  0.5
R[i,j] 2.7964186266624713
low thres  0.5
R[i,j] 3.052212

R[i,j] 2.5343424447339147
low thres  0.5
R[i,j] 2.6668143469402685
low thres  0.5
R[i,j] 2.6171898865614605
low thres  0.5
R[i,j] 2.9059134383165977
low thres  0.5
R[i,j] 3.6439159829922603
low thres  0.5
R[i,j] 2.366501295171793
low thres  0.5
R[i,j] 2.578564323837016
low thres  0.5
R[i,j] 2.764835539149565
low thres  0.5
R[i,j] 4.011395353737709
low thres  0.5
R[i,j] 2.4368395412267128
low thres  0.5
R[i,j] 2.401361749624421
low thres  0.5
R[i,j] 4.35734988887781
low thres  0.5
R[i,j] 2.9306424167269607
low thres  0.5
R[i,j] 2.364389655053901
low thres  0.5
R[i,j] 2.838976876267119
low thres  0.5
R[i,j] 2.692796577861552
low thres  0.5
R[i,j] 2.5370629527970894
low thres  0.5
R[i,j] 3.7189251956415967
low thres  0.5
R[i,j] 2.824185378756763
low thres  0.5
R[i,j] 2.7837755840796725
low thres  0.5
R[i,j] 5.867060613558904
low thres  0.5
R[i,j] 2.4968384233574383
low thres  0.5
R[i,j] 3.4034018555729713
low thres  0.5
R[i,j] 2.824311729272001
low thres  0.5
R[i,j] 3.015927458329313
low 

R[i,j] 2.4736902993286765
low thres  0.5
R[i,j] 2.8782333958851476
low thres  0.5
R[i,j] 2.3815057473643515
low thres  0.5
R[i,j] 3.7511263336955447
low thres  0.5
R[i,j] 2.462843572981686
low thres  0.5
R[i,j] 2.5908413994955146
low thres  0.5
R[i,j] 2.6571907786228652
low thres  0.5
R[i,j] 3.507664519623909
low thres  0.5
R[i,j] 5.089978969667461
low thres  0.5
R[i,j] 2.4319763349453107
low thres  0.5
R[i,j] 2.5810235234194194
low thres  0.5
R[i,j] 2.6044780691535334
low thres  0.5
R[i,j] 2.35421292072834
low thres  0.5
R[i,j] 2.5814505164117234
low thres  0.5
R[i,j] 3.1683345955560265
low thres  0.5
R[i,j] 2.689076080427016
low thres  0.5
R[i,j] 4.058863718010971
low thres  0.5
R[i,j] 3.5679597422616594
low thres  0.5
R[i,j] 2.6530785166496536
low thres  0.5
R[i,j] 2.825919216136868
low thres  0.5
R[i,j] 2.538942909323279
low thres  0.5
R[i,j] 2.4766577395441405
low thres  0.5
R[i,j] 2.5985821544926417
low thres  0.5
R[i,j] 5.014709553300544
low thres  0.5
R[i,j] 2.5603498900813744
